In [9]:
import numpy as np
import pandas as pd
import csv
import requests
import json
import datetime
import html

api_key = 'sk_f491c9326e6a4b79a7ed5b1689c178f3'
csv_read = csv.reader(open('companylist_all.csv'))
symbols = []
symbol_dict = {}
from bs4 import BeautifulSoup as bs
for i in csv_read:
    if i[0] != 'Symbol' :
        symbols.append(i[0])
        
        
x = len(symbols)
symbols = ['AAPL']
for symbol in symbols: 
    try:
        url = requests.get(f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}')
        soup = bs(url.content,'html.parser')
        scripts = soup.find_all('script')
        fin_data = scripts[52]
        fin_data = str(fin_data).split('root.App.main = ')[1]
        fin_data = fin_data.split(';\n}(this));\n</script>')[0]
        fin_dict = json.loads(fin_data)
        historical_prices = fin_dict['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
        df = pd.DataFrame(historical_prices)
        df.sort_index(ascending=False,inplace=True)
        df['date'] = pd.to_datetime(df['date'],unit='s')
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y %H:%M:%S').dt.strftime('%m/%d/%Y')
        df.set_index('date', inplace = True)
        df.dropna(subset =['open'],inplace=True)
        df[['adjclose','open','close','high','low','open']] = df[['adjclose','open','close','high','low','open']].round(2)
        df['open'] = df['open'].astype(float)
        df['high'] = df['high'].astype(float)
        df['low'] = df['low'].astype(float)
        df['close'] = df['close'].astype(float)
        df['volume'] = df['volume'].astype(float)
        df['fast'] = df['close'].ewm(span=12).mean()
        df['slow'] = df['close'].ewm(span=26).mean()
        df['macd'] = df['fast']-df['slow']
        df['signal'] = df['macd'].ewm(span=9).mean()
        df['200MA'] = df['close'].rolling(200).mean()
        df['50MA'] = df['close'].rolling(50).mean()
        df['100MA'] = df['close'].rolling(100).mean()
        df['10MA'] = df['close'].rolling(10).mean()
        df['20MA'] = df['close'].rolling(20).mean()
        df['BB_20_upper_2std'] = (df['close'].rolling(20).std()*2) + df['20MA']
        df['BB_20_lower_2std'] = df['20MA'] - (df['close'].rolling(20).std()*2)
        df['day_change'] = df['close'].diff(periods=1)
        df['2_day_change'] = df['close'].diff(periods=2)
        df['3_day_change'] = df['close'].diff(periods=3)
        df['5_day_change'] = df['close'].diff(periods=5)
        symbol_dict[symbol] = df
    except:
        print('Symbol Lookup Fail')
        pass
    print(x)
    x -= 1
# for symbol in symbol_dict: 
#     df = symbol_dict[symbol]
#     try:
#         if (df['3_day_change'].iloc[-1]/df['close'].iloc[-4]) > 2 and df['volume'].iloc[-1] >1000000:
#             print(symbol, df['close'].iloc[-4],df['close'].iloc[-1])
#     except:
#         pass
    
for symbol in symbol_dict: 
    df = symbol_dict[symbol]
    try: 
        if (df['20MA'].iloc[-1] < df['200MA'].iloc[-1] or df['20MA'].iloc[-1] < df['100MA'].iloc[-1]) and df['50MA'].iloc[-1] > df['100MA'].iloc[-1] and df['100MA'].iloc[-1] > df['200MA'].iloc[-1]:
            print(symbol,df['close'].iloc[-1])
    except:
        pass


Symbol Lookup Fail
6895


In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import json

symbol = 'AAPL'
url = requests.get(f'https://finance.yahoo.com/quote/{symbol}/history')
soup = bs(url.content,'html.parser')
scripts = soup.find_all('script')
fin_data = scripts[53]
fin_data = str(fin_data).split('root.App.main = ')[1]
fin_data = fin_data.split(';\n}(this));\n</script>')[0]
fin_dict = json.loads(fin_data)
historical_prices = fin_dict['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
df = pd.DataFrame(historical_prices)
df.sort_index(ascending=False,inplace=True)
df['date'] = pd.to_datetime(df['date'],unit='s')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y %H:%M:%S').dt.strftime('%m/%d/%Y')
df.set_index('date', inplace = True)
df.dropna(subset =['open'],inplace=True)
df[['adjclose','open','close','high','low','open']] = df[['adjclose','open','close','high','low','open']].round(2)
df.tail()

IndexError: list index out of range

In [8]:
scripts

[<script>
     document.write('<img src="//geo.yahoo.com/b?s=1197757129&t='+new Date().getTime()+'&src=aws&err_url='+encodeURIComponent(document.URL)+'&err=%<pssc>&test='+encodeURIComponent('%<{Bucket}cqh[:200]>')+'" width="0px" height="0px"/>');var beacon = new Image();beacon.src="//bcn.fp.yahoo.com/p?s=1197757129&t="+new Date().getTime()+"&src=aws&err_url="+encodeURIComponent(document.URL)+"&err=%<pssc>&test="+encodeURIComponent('%<{Bucket}cqh[:200]>');
   </script>]

In [4]:


for symbol in symbol_dict: 
    df = symbol_dict[symbol]
    try:
        if (df['3_day_change'].iloc[-1]/df['close'].iloc[-4]) > 1 and df['volume'].iloc[-1] >1000000:
            print(symbol, df['close'].iloc[-4],df['close'].iloc[-1])
    except:
        pass

c:\users\rob\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


SSI 0.03 14900.0
GRNQ 0.36 2.58
ZIONW 0.17 0.44


In [10]:
soup

<!DOCTYPE html>

<html lang="en-us"><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<title>Yahoo</title>
<meta content="width=device-width,initial-scale=1,minimal-ui" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<style>
  html {
      height: 100%;
  }
  body {
      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
      background-size: cover;
      height: 100%;
      text-align: center;
      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;
  }
  table {
      height: 100%;
      width: 100%;
      table-layout: fixed;
      border-collapse: collapse;
      border-spacing: 0;
      border: none;
  }
  h1 {
      font-size: 42px;
      font-weight: 400;
      color: #400090;
  }
  p {
      color: #1A1A1A;
  }
  #message-1 {
      font-weight: bold;
      margin: 0;
  }
  #message-2 {
      display: inline-block;
      *display: